## siRNA knockdown ##

This notebook fits analytical functions to Rafał’s data.

The analytical functions look like this:

$$f_\text{red}(t) =
m_\text{r}\,k_\text{tl} \left(
\frac{1}{\beta_\text{r}-\delta_\text{r}+k_\text{m,r}} \mathrm{e}^{-(\beta_\text{r}+k_\text{m,r})(t-t_0)}
-\frac{1}{\beta_\text{r} - \delta_\text{r}} \mathrm{e}^{-\beta_\text{r} (t-t_0)}
+\frac{k_\text{m,r}}{(\beta_\text{r}-\delta_\text{r}) (\beta_\text{r}-\delta_\text{r}+k_\text{m,r})} \mathrm{e}^{-\delta_\text{r} (t-t_0)}
\right)
$$

$$f_\text{green}(t) =
m_\text{g}\,k_\text{tl} \left(
\frac{1}{\beta_\text{g}-\delta_\text{g}+k_\text{m,g}} \mathrm{e}^{-(\beta_\text{g}+k_\text{m,g})(t-t_0)}
-\frac{1}{\beta_\text{g} - \delta_\text{g}} \mathrm{e}^{-\beta_\text{g} (t-t_0)}
+\frac{k_\text{m,g}}{(\beta_\text{g}-\delta_\text{g}) (\beta_\text{g}-\delta_\text{g}+k_\text{m,g})} \mathrm{e}^{-\delta_\text{g} (t-t_0)}
\right)
$$

The notebook has the following structure:

At first, the model functions are defined and the data is loaded. The next section contains code for fitting the two models separately. The next section contains code for fitting the two traces in one run with parameters shared among the models.

The section of the combined model also provides a function for a “second run”. The idea of the second run was to fit the traces in a second run, with all datapoints before the onset time estimated in the first run ignored. This was intended to compensate for possible bias by measurement background. However, it does not yield a satisfying effect. Note that the second run fit overwrites the results of the first fit in `R`.

In [ ]:
# Import modules needed
%matplotlib inline
import numpy as np
np.seterr(divide='print')
import scipy as sc
import lmfit as lm
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
import pandas as pd
from io_Daniel import *
import os
import pickle
import ipywidgets as wdg
import IPython

In [ ]:
def red(t, tr, m_ktl, kmr, betr, deltr, offr):
    """Model function for red data"""

    f = np.zeros(np.shape(t))
    idx_after = (t > tr)
    dt = t[idx_after] - tr

    f1 = np.exp(- (betr + kmr) * dt) / (betr - deltr + kmr)
    f2 = - np.exp(- betr * dt) / (betr - deltr)
    f3 = kmr * np.exp(- deltr * dt) / (betr - deltr) / (betr - deltr + kmr)

    f[idx_after] = (f1 + f2 + f3) * m_ktl

    return f + offr

In [ ]:
def green(t, tg, m_ktl, kmg, betg, deltg, offg):
    """Model function for green data"""

    f = np.zeros(np.shape(t))
    idx_after = t > tg
    dt = t[idx_after] - tg

    f1 = np.exp(- (betg + kmg) * dt) / (betg - deltg + kmg)
    f2 = - np.exp(- betg * dt) / (betg - deltg)
    f3 = kmg * np.exp(- deltg * dt) / (betg - deltg) / (betg - deltg + kmg)

    f[idx_after] = (f1 + f2 + f3) * m_ktl

    return f + offg

In [ ]:
def combined(t, tr, tg, m_ktl, kmr, kmg, betr, betg, deltr, deltg, offr, offg):
    """Model function for a combined fit of red and green data"""

    f = np.stack(
        (red(t=t, tr=tr, m_ktl=m_ktl, kmr=kmr, betr=betr, deltr=deltr, offr=offr),
         green(t=t, tg=tg, m_ktl=m_ktl, kmg=kmg, betg=betg, deltg=deltg, offg=offg)),
        axis=1)

    return f

In [ ]:
# Set default parameter values
m_ktl_0 = 10

tr_0 = 4
kmr_0 = 0.2
betr_0 = 0.1
deltr_0 = 10
offr_0 = 0

tg_0 = 3
kmg_0 = 1
betg_0 = 0.01
deltg_0 = 50
offg_0 = 0

In [ ]:
# Create fit models

# Separate fit of red trace
model_red = lm.Model(red)
model_red.set_param_hint('tr', min=0, max=30, value=tr_0)
model_red.set_param_hint('m_ktl', min=0, value=m_ktl_0)
model_red.set_param_hint('kmr', min=0, value=kmr_0)
model_red.set_param_hint('betr', min=0, value=betr_0)
model_red.set_param_hint('deltr', min=0, value=deltr_0)
model_red.set_param_hint('offr', value=offr_0)

# Separate fit of green trace
model_green = lm.Model(green)
model_green.set_param_hint('tg', min=0, max=30, value=tg_0)
model_green.set_param_hint('m_ktl', min=0, value=m_ktl_0)
model_green.set_param_hint('kmg', min=0, value=kmg_0)
model_green.set_param_hint('betg', min=0, value=betg_0)
model_green.set_param_hint('deltg', min=0, value=deltg_0)
model_green.set_param_hint('offg', value=offg_0)

# Combined fit of red and green trace (first run)
model_combined = lm.Model(combined)
model_combined.set_param_hint('tr', min=0, max=30, value=tr_0)
model_combined.set_param_hint('tg', min=0, max=30, value=tg_0)
model_combined.set_param_hint('m_ktl', min=0, value=m_ktl_0)
model_combined.set_param_hint('kmr', min=0, max=.3, value=kmr_0)
model_combined.set_param_hint('kmg', min=0, max=1.5, value=kmg_0)
model_combined.set_param_hint('betr', min=0, max=.6, value=betr_0)
model_combined.set_param_hint('betg', min=0, max=.03, value=betg_0)
model_combined.set_param_hint('deltr', min=0, value=deltr_0)
model_combined.set_param_hint('deltg', min=0, value=deltg_0)
model_combined.set_param_hint('offr', value=offr_0)
model_combined.set_param_hint('offg', value=offg_0)

# Combined fit of red and green trace (second run)
params2 = lm.Parameters()
params2.add('tr', min=0, max=30, value=tr_0)
params2.add('tg', min=0, max=30, value=tg_0)
params2.add('m_ktl', min=0, value=m_ktl_0)
params2.add('kmr', min=0, value=kmr_0)
params2.add('kmg', min=0, value=kmg_0)
params2.add('betr', min=0, value=betr_0)
params2.add('betg', min=0, value=betg_0)
params2.add('deltr', min=0, value=deltr_0)
params2.add('deltg', min=0, value=deltg_0)
params2.add('offr', value=offr_0)
params2.add('offg', value=offg_0)

## Read in data and prepare result list

In [ ]:
def plotViolin(ax, data, label, clr_face, clr_edge, mark=None, showext=False):
    """Plots the current parameter value in realtion to the values for the whole dataset"""
    v = ax.violinplot(data, showextrema=showext, positions=[0])
    for p in v.pop('bodies'):
        p.set_facecolor(clr_face)
    if showext:
        for p in v.values():
            p.set_edgecolor(clr_edge)
    if mark != None:
        ax.plot(0, mark, 'x', color=clr_edge)
    ax.set_xticks([])
    ax.spines['left'].set_position('zero')
    for s in [ax.spines[pos] for pos in ['top', 'right', 'bottom']]:
        s.set_visible(False)
    ax.set_title(label)

In [ ]:
def index_mask(n, idcs=[], negative: bool=True):
    """Returns a logical vector for indexing an array.

    Keyword arguments:
    n -- the length of the index mask
    idcs -- indices to be excluded from the mask
    negative -- boolean deciding whether to select all mask indices but those in idcs, or only the indices in idcs
    """
    mask = np.empty(n, dtype='bool_')
    mask.fill(negative)
    mask[idcs] = not negative

    return mask

In [ ]:
# Read in data (OLD VERSION!)
D = {}

filenames = {
    'A549 siRNA': 'data/seq3_A549_siRNA_#molecules.xlsx',
    'A549 control': 'data/seq4_A549_control_#molecules.xlsx',
    'Huh7 siRNA': 'data/seq10_Huh7_siRNA_#molecules.xlsx',
    'Huh7 control': 'data/seq6_Huh7_control_#molecules.xlsx'
}

for condition, filename in filenames.items():
    # Read sheets from excel file
    X = pd.read_excel(filename, sheetname=['#RFP', '#GFP', '#GFP_corrected'])

    # Write data into easy-to-access structure
    D[condition] = {}
    D[condition]['t'] = X['#RFP'].values[:,0].flatten()
    D[condition]['rfp'] = X['#RFP'].values[:,1:]
    D[condition]['gfp'] = X['#GFP'].values[:,1:]
    D[condition]['gfp_corr'] = X['#GFP_corrected'].values[:,1:]

    # Shift values to positive interval
    for q in ('rfp', 'gfp', 'gfp_corr'):
        min_vals = D[condition][q].min(axis=0)
        for i in range(np.shape(D[condition][q])[1]):
            D[condition][q][:,i] -= min_vals[i]

In [ ]:
# Prepare data loading

# Define available files
datafiles = [
    {
        "sample": "A549",
        "condition": "siRNA",
        "measurement": "2016-01-09_seq3",
        "file": "data/2016-01-09_seq3_A549_siRNA_#molecules.xlsx"
    }, {
        "sample": "A549",
        "condition": "control",
        "measurement": "2016-01-09_seq5",
        "file": "data/2016-01-09_seq5_A549_Control_#molecules.xlsx"
    }, {
        "sample": "A549",
        "condition": "siRNA",
        "measurement": "2016-12-20_seq3",
        "file": "data/2016-12-20_seq3_A549_siRNA_#molecules.xlsx"
    }, {
        "sample": "A549",
        "condition": "control",
        "measurement": "2016-12-20_seq4",
        "file": "data/2016-12-20_seq4_A549_control_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "siRNA",
        "measurement": "2017-05-26_seq10",
        "file": "data/2017-05-26_seq10_Huh7_siRNA_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "siRNA",
        "measurement": "2017-05-26_seq11",
        "file": "data/2017-05-26_seq11_Huh7_siRNA_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "control",
        "measurement": "2017-05-26_seq6",
        "file": "data/2017-05-26_seq6_Huh7_control_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "control",
        "measurement": "2017-05-26_seq7",
        "file": "data/2017-05-26_seq7_Huh7_control_#molecules.xlsx"
    }
]

# By default, mark all files for loading
load_idcs = range(len(datafiles))

# Define function for loading data
def load_data_from_files():
    global D
    D = []
    for i in load_idcs:
        # Show message
        print("Loading file: {}".format(datafiles[i]["file"]))

        # Read sheets from excel file
        X = pd.read_excel(datafiles[i]['file'], sheetname=[
            '#RFP', '#GFP', '#GFP_corrected', '#RFP_error', '#GFP_error'])

        # Write data into easy-to-access structure
        d = {}
        d['sample'] = datafiles[i]['sample']
        d['condition'] = datafiles[i]['condition']
        d['measurement'] = datafiles[i]['measurement']
        d['file'] = datafiles[i]['file']
        d['t'] = X['#RFP'].values[:,0].flatten()
        d['rfp'] = X['#RFP'].values[:,1:]
        d['gfp'] = X['#GFP'].values[:,1:]
        d['gfp_corr'] = X['#GFP_corrected'].values[:,1:]
        d['rfp_error'] = X['#RFP_error'].values[:,1:]
        d['gfp_error'] = X['#GFP_error'].values[:,1:]
        D.append(d)

In [ ]:
# Read in data (new version)

# Prompt user for files to load
lbl = wdg.Label('Select the files to load:')
lbl.layout.width = 'initial'
entries = []
for f in datafiles:
    entries.append("{} {}: {}".format(
        f['sample'], f['condition'], f['file']))
sel_entry = wdg.SelectMultiple(options=entries, rows=len(entries))
sel_entry.layout.width = 'initial'
bload = wdg.Button(description='Load')
bselall = wdg.Button(description='Select all')
bselnone = wdg.Button(description='Select none')

# Define callbacks
def sel_all_files(_):
    sel_entry.value = entries
def sel_no_files(_):
    sel_entry.value = ()
def load_button_clicked(_):
    global load_idcs
    load_idcs = [entries.index(r) for r in sel_entry.value]
    vb.close()
    load_data_from_files()
bselall.on_click(sel_all_files)
bselnone.on_click(sel_no_files)
bload.on_click(load_button_clicked)

# Finally, show the widgets
vb = wdg.VBox((lbl, sel_entry, wdg.HBox((bload,bselall,bselnone))))
IPython.display.display(vb)

In [ ]:
# Provide output tables

# Initialize result dictionary
R = []

# Get a list of fit parameters
par_names = model_green.param_names.copy()
par_names.extend(p for p in model_red.param_names if p not in par_names)
par_names.sort()

# Iteratively populate the result dictionary
for k in range(len(D)):
    R[k] = {}
    nTraces = np.shape(D[k]['gfp'])[1]
    nTimes = np.shape(D[k]['gfp'])[0]
    tpl_traces = np.empty((nTimes, nTraces))
    tpl_traces.fill(np.NaN)

    R[k]['green'] = {}
    R[k]['green']['params'] = pd.DataFrame(index=np.arange(nTraces), columns=model_green.param_names, dtype='float64')
    R[k]['green']['fit'] = np.copy(tpl_traces)

    R[k]['red'] = {}
    R[k]['red']['params'] = pd.DataFrame(index=np.arange(nTraces), columns=model_red.param_names, dtype='float64')
    R[k]['red']['fit'] = np.copy(tpl_traces)

    R[k]['combined'] = {}
    R[k]['combined']['params'] = pd.DataFrame(index=np.arange(nTraces), columns=model_combined.param_names, dtype='float64')
    #R[k]['combined']['fit'] = {}

In [ ]:
# Test for negative parameter values
for ds in sorted(R.keys()):
    for method in sorted(R[ds].keys()):
        print('Negative parameter values in “{}” ({}): {}'.format(
            ds, method, np.any(R[ds][method]['params'].values < 0)))

In [ ]:
# Pickle fit results for future sessions
outfile = getTimeStamp() + '_fit_results.pickled'
with open(outfile, 'wb') as f:
    pickle.dump(R, f)

In [ ]:
# Load pickled results (requires file suffix “.pickled”)
pickfiles = [f for f in os.listdir() if f.lower().endswith('.pickled')]
pickfiles.sort(reverse=True)

lbl = wdg.Label('Select the file to load:')
lbl.layout.width = 'initial'
rad = wdg.RadioButtons(options=pickfiles)
but = wdg.Button(description='Load')
vb = wdg.VBox([lbl, rad, but])
IPython.display.display(vb)

def clicked_on_but(b):
    global R
    with open(rad.value, 'rb') as f:
        R = pickle.load(f)
    print('Loaded: ' + rad.value)
    vb.close()
but.on_click(clicked_on_but)

## Fit and plot separate models

In [ ]:
def plotSeparate(ds, tr, pdf=None, params=False):
    """Fits and plots the data, treating RFP and GFP separately.

    Keyword arguments:
    ds -- the dictionary key of the dataset
    tr -- the index of the trace in the dataset to be processed
    pdf -- a PdfPages object to which the figure is written if it is not None
    params -- if set to True, the parameters will be shown
    """

    # Plot fit results
    fig = plt.figure()

    if params:
        fig.set_figwidth(1.6 * fig.get_figwidth())

        pn_red = ['m_ktl', 'tr', 'kmr', 'betr', 'deltr', 'offr']
        pn_green = ['m_ktl', 'tg', 'kmg', 'betg', 'deltg', 'offg']

        grid = (2, 1+max(len(pn_red), len(pn_green)))
        wr = [grid[1]] + ( [1] * (grid[1] - 1) )
        gs = GridSpec(grid[0], grid[1], width_ratios=wr)

        # Plot green parameters
        for prm in range(len(pn_green)):
            ax = plt.subplot(gs.new_subplotspec((0, prm+1)))
            label = pn_green[prm]
            data = R[ds]['combined']['params'][label].values
            clr_face = '#00ff0055'
            clr_edge = '#009900ff'
            plotViolin(ax, data, label, clr_face, clr_edge, data[tr])

        # Plot red parameters
        for prm in range(len(pn_red)):
            ax = plt.subplot(gs.new_subplotspec((1, prm+1)))
            label = pn_red[prm]
            data = R[ds]['combined']['params'][label].values
            clr_face = '#ff000055'
            clr_edge = '#990000ff'
            plotViolin(ax, data, label, clr_face, clr_edge, data[tr])
        
        ax = plt.subplot(gs.new_subplotspec((0, 0), rowspan=2))

    else:
        ax = fig.gca()

    p_tr = ax.axvline(R[ds]['red']['params']['tr'][tr], label='RFP onset',
                       color='#ff0000', linewidth=.5, linestyle='--')
    p_tg = ax.axvline(R[ds]['green']['params']['tg'][tr], label='GFP onset',
                      color='#00ff00', linewidth=.5, linestyle='--')
    p_fr, = ax.plot(D[ds]['t'], R[ds]['red']['fit'][:,tr], '-', label='RFP (fit)', color='#ff0000', linewidth=1)
    p_fg, = ax.plot(D[ds]['t'], R[ds]['green']['fit'][:,tr], '-', label='GFP (fit)', color='#00ff00', linewidth=1)
    p_dr, = ax.plot(D[ds]['t'], D[ds]['rfp'][:,tr], '-', label='RFP (measured)', color='#990000', linewidth=.5)
    p_dg, = ax.plot(D[ds]['t'], D[ds]['gfp'][:,tr], '-', label='GFP (measured)', color='#009900', linewidth=.5)

    # Format plot
    ax.set_xlabel('Time [h]')
    ax.set_ylabel('Fluorescence intensity [a.u.]')
    ax.set_title('{:s} (separate fit) #{:03d}'.format(ds, tr))
    ax.legend(handles=[p_dg, p_fg, p_tg, p_dr, p_fr, p_tr])

    # Write figure to pdf
    if pdf != None:
        pdf.savefig(fig)

    # Show and close figure
    plt.show(fig)
    plt.close(fig)

In [ ]:
# Fit traces separately
for ds in D.keys():
    nTraces = np.shape(D[ds]['rfp'])[1]

    for tr in range(nTraces):
        print('Fitting „{:s}“ #{:03d}/{:03d} …'.format(ds, tr, nTraces))

        # Adjust parameter bounds for onset time for current trace
        model_red.set_param_hint('tr', max=D[ds]['t'][D[ds]['rfp'][:,tr].argmax()])
        model_green.set_param_hint('tg', max=D[ds]['t'][D[ds]['gfp'][:,tr].argmax()])

        # Fit the data
        data_red = D[ds]['rfp'][:,tr]
        data_green = D[ds]['gfp'][:,tr]
        wght_red = 1 / np.sqrt(data_red)
        wght_green = 1 / np.sqrt(data_green)
        result_red = model_red.fit(data_red, t=D[ds]['t'], weights=wght_red)
        result_green = model_green.fit(data_green, t=D[ds]['t'], weights=wght_green)

        # Save results to R
        R[ds]['red']['params'].iloc[tr] = result_red.best_values
        R[ds]['red']['fit'][:,tr] = result_red.best_fit
        R[ds]['green']['params'].iloc[tr] = result_green.best_values
        R[ds]['green']['fit'][:,tr] = result_green.best_fit

In [ ]:
# Plot results of separate fit
ts = getTimeStamp()

for ds in D.keys():
    pdffile = os.path.join(getOutpath(), '{:s}_separate_{:s}.pdf'.format(ts, ds.replace(' ', '_')))
    with PdfPages(pdffile) as pdf:
        for tr in range(np.shape(D[ds]['rfp'])[1]):
            plotSeparate(ds, tr, pdf, True)

## Fit and plot combined model

In [ ]:
# Define the objective function for second run
def objectiveCombined2(params, data_red, data_green, times_red, times_green):
    """Objective function for fitting the traces with any values before the onset cut"""
    P = params.valuesdict()

    fitted_red = red(t=times_red, **{k: P[k] for k in ['tr', 'm_ktl', 'kmr', 'betr', 'deltr', 'offr']})
    fitted_green = green(t=times_green, **{k: P[k] for k in ['tg', 'm_ktl', 'kmg', 'betg', 'deltg', 'offg']})

    data = np.concatenate((data_red, data_green))
    weights = 1 / np.sqrt(data)

    return weights * (np.concatenate((fitted_red, fitted_green)) - data)

In [ ]:
def plotCombined(ds, tr, pdf=None, params=False):
    """Fits and plots the data, treating RFP and GFP together.
    
    Keyword arguments:
    ds -- the dictionary key of the dataset
    tr -- the index of the trace in the dataset to be processed
    pdf -- a PdfPages object to which the figure is written if it is not None
    params -- if set to True, the parameters will be shown
    """

    # Plot fit results
    fig = plt.figure()

    if params:
        fig.set_figwidth(1.6 * fig.get_figwidth())

        #pn_both = ['m', 'ktl']
        pn_both = ['m_ktl']
        pn_red = ['tr', 'kmr', 'betr', 'deltr', 'offr']
        pn_green = ['tg', 'kmg', 'betg', 'deltg', 'offg']

        grid = (2, 2+max(len(pn_red), len(pn_green)))
        wr = [grid[1]] + ( [1] * (grid[1] - 1) )
        gs = GridSpec(grid[0], grid[1], width_ratios=wr)
        

        # Plot combined parameters
        #for prm in range(len(pn_both)):
        prm = 0
        ax = plt.subplot(gs.new_subplotspec((prm, 1), rowspan=2))
        label = pn_both[prm]
        data = R[ds]['combined']['params'][label].values
        clr_face = '#0000ff55'
        clr_edge = '#000099ff'
        plotViolin(ax, data, label, clr_face, clr_edge, data[tr])

        # Plot green parameters
        for prm in range(len(pn_green)):
            ax = plt.subplot(gs.new_subplotspec((0, prm+2)))
            label = pn_green[prm]
            data = R[ds]['combined']['params'][label].values
            clr_face = '#00ff0055'
            clr_edge = '#009900ff'
            plotViolin(ax, data, label, clr_face, clr_edge, data[tr])

        # Plot red parameters
        for prm in range(len(pn_red)):
            ax = plt.subplot(gs.new_subplotspec((1, prm+2)))
            label = pn_red[prm]
            data = R[ds]['combined']['params'][label].values
            clr_face = '#ff000055'
            clr_edge = '#990000ff'
            plotViolin(ax, data, label, clr_face, clr_edge, data[tr])
        
        ax = plt.subplot(gs.new_subplotspec((0, 0), rowspan=2))

    else:
        ax = fig.gca()

    #wr = np.sqrt(D[ds]['rfp'][:,tr])
    #ax.fill_between(D[ds]['t'], D[ds]['rfp'][:,tr]-wr, D[ds]['rfp'][:,tr]+wr, color='#ff000033')
    #wg = np.sqrt(D[ds]['gfp'][:,tr])
    #ax.fill_between(D[ds]['t'], D[ds]['gfp'][:,tr]-wg, D[ds]['gfp'][:,tr]+wg, color='#00ff0033')

    p_tr = ax.axvline(R[ds]['combined']['params']['tr'][tr], label='RFP onset',
                       color='#ff0000', linewidth=.5, linestyle='--')
    p_tg = ax.axvline(R[ds]['combined']['params']['tg'][tr], label='GFP onset',
                      color='#00ff00', linewidth=.5, linestyle='--')
    p_fr, = ax.plot(D[ds]['t'], R[ds]['combined']['fit']['red'][tr], '-', label='RFP (fit)', color='#ff0000', linewidth=1)
    p_fg, = ax.plot(D[ds]['t'], R[ds]['combined']['fit']['green'][tr], '-', label='GFP (fit)', color='#00ff00', linewidth=1)
    p_dr, = ax.plot(D[ds]['t'], D[ds]['rfp'][:,tr], '-', label='RFP (measured)', color='#990000', linewidth=.5)
    p_dg, = ax.plot(D[ds]['t'], D[ds]['gfp'][:,tr], '-', label='GFP (measured)', color='#009900', linewidth=.5)

    # Format plot
    ax.set_xlabel('Time [h]')
    ax.set_ylabel('Fluorescence intensity [a.u.]')
    ax.set_title('{} {} [{}] (combined fit) #{:03d}'.format(
        D[ds]['sample'], D[ds]['condition'], D[ds]['measurement'], tr))
    ax.legend(handles=[p_dg, p_fg, p_tg, p_dr, p_fr, p_tr])

    # Write figure to pdf
    if pdf != None:
        pdf.savefig(fig)

    # Show and close figure
    plt.show(fig)
    plt.close(fig)

In [ ]:
# Fit combined model
for ds in D.keys():
    R[ds]['combined']['fit'] = {'red': [], 'green': []}
    nTraces = np.shape(D[ds]['rfp'])[1]

    for tr in range(nTraces):
        print('Fitting „{:s}“ #{:03d}/{:03d} …'.format(ds, tr, nTraces))

        # Adjust parameter bounds for onset time for current trace
        model_combined.set_param_hint('tr', max=D[ds]['t'][D[ds]['rfp'][:,tr].argmax()])
        model_combined.set_param_hint('tg', max=D[ds]['t'][D[ds]['gfp'][:,tr].argmax()])

        # Fit the data
        data = np.stack([D[ds]['rfp'][:,tr], D[ds]['gfp'][:,tr]], axis=1)
        wght = 1 / np.sqrt(data - data.min(axis=0))
        wght_inf = np.nonzero(np.isinf(wght))
        wght[wght_inf[0], wght_inf[1]] = wght.min(axis=0)[wght_inf[1]]
        if not np.all(np.isfinite(wght)):
            raise ValueError("Bad weight encountered in D['{}'] (Trace {}):\n{}".format(ds, tr, wght))
        result = model_combined.fit(data, t=D[ds]['t'], weights=wght)

        # Save results to R
        R[ds]['combined']['params'].iloc[tr] = result.best_values
        R[ds]['combined']['fit']['red'].insert(tr, result.best_fit[:,0])
        R[ds]['combined']['fit']['green'].insert(tr, result.best_fit[:,1])

In [ ]:
# Fit second run (combined model)
for ds in D.keys():
    R[ds]['combined']['fit'] = {'red': [], 'green': []}
    nTraces = np.shape(D[ds]['rfp'])[1]

    for tr in range(nTraces):
        print('Fitting „{:s}“ #{:03d}/{:03d} …'.format(ds, tr, nTraces))

        # Adjust parameter bounds for onset time for current trace
        params2['tr'].set(max=D[ds]['t'][D[ds]['rfp'][:,tr].argmax()])
        params2['tg'].set(max=D[ds]['t'][D[ds]['gfp'][:,tr].argmax()])

        # Cut current trace before onset
        time = D[ds]['t']

        fitidcs_red = time > R[ds]['combined']['params']['tr'].iloc[tr]
        fitidcs_green = time > R[ds]['combined']['params']['tg'].iloc[tr]

        time_red = time[fitidcs_red]
        time_green = time[fitidcs_green]

        data_red = D[ds]['rfp'][fitidcs_red, tr]
        data_green = D[ds]['gfp'][fitidcs_green, tr]

        # Fit the data
        wght = [1/np.sqrt(data[0]), 1/np.sqrt(data[1])]
        result = lm.minimize(objectiveCombined2, params2, kws={
            'data_red': data_red,
            'data_green': data_green,
            'times_red': time_red,
            'times_green': time_green})

        best_values = result.params.valuesdict()
        best_fit_green = green(t=time, **{k: best_values[k] for k in ['tg', 'm_ktl', 'kmg', 'betg', 'deltg', 'offg']})
        best_fit_red = red(t=time, **{k: best_values[k] for k in ['tr', 'm_ktl', 'kmr', 'betr', 'deltr', 'offr']})

        # Save results to R
        R[ds]['combined']['params'].iloc[tr] = best_values
        R[ds]['combined']['fit']['red'].insert(tr, best_fit_red)
        R[ds]['combined']['fit']['green'].insert(tr, best_fit_green)

In [ ]:
# Plot results of combined fit
ts = getTimeStamp()
for ds in D.keys():
    pdffile = os.path.join(getOutpath(), '{:s}_combined_{:s}.pdf'.format(ts, ds.replace(' ', '_')))
    with PdfPages(pdffile) as pdf:
        for tr in range(np.shape(D[ds]['rfp'])[1]):
            plotCombined(ds, tr, pdf, params=True)

In [ ]:
# Plot violin distributions of the data sets
#pn_both = ['m', 'ktl']
pn_both = ['m_ktl']
pn_red = ['tr', 'kmr', 'betr', 'deltr', 'offr']
pn_green = ['tg', 'kmg', 'betg', 'deltg', 'offg']

grid = (2, 1+max(len(pn_red), len(pn_green)))

with PdfPages(os.path.join(getOutpath(), '{:s}_parameter_violins.pdf'.format(getTimeStamp()))) as pdf:
    for ds in sorted(D.keys()):
        fig = plt.figure()
        gs = GridSpec(grid[0], grid[1])

        # Plot combined parameters
        #for prm in range(len(pn_both)):
        prm = 0
        ax = plt.subplot(gs.new_subplotspec((prm, 0), rowspan=2))
        label = pn_both[prm]
        data = R[ds]['combined']['params'][label].values
        clr_face = '#0000ffff'
        clr_edge = '#000099ff'
        plotViolin(ax, data, label, clr_face, clr_edge)

        # Plot green parameters
        for prm in range(len(pn_green)):
            ax = plt.subplot(gs.new_subplotspec((0, prm+1)))
            label = pn_green[prm]
            data = R[ds]['combined']['params'][label].values
            clr_face = '#00ff00ff'
            clr_edge = '#009900ff'
            plotViolin(ax, data, label, clr_face, clr_edge)

        # Plot red parameters
        for prm in range(len(pn_red)):
            ax = plt.subplot(gs.new_subplotspec((1, prm+1)))
            label = pn_red[prm]
            data = R[ds]['combined']['params'][label].values
            clr_face = '#ff0000ff'
            clr_edge = '#990000ff'
            plotViolin(ax, data, label, clr_face, clr_edge)

        # Show and close figure
        fig.suptitle(ds)
        pdf.savefig(fig)
        plt.show(fig)
        plt.close(fig)

In [ ]:
# Plot the parameter distributions for the datasets
ds_keys = list(R.keys())
ds_keys.sort()
params = R[ds_keys[0]]['combined']['params'].columns
grid = (len(params), len(ds_keys))
i_col = 0

pdffile = os.path.join(getOutpath(), '{:s}_parameters.pdf'.format(getTimeStamp()))
with PdfPages(pdffile) as pdf:
    fig = plt.figure()
    fig.set_figheight(grid[0] * .8 * fig.get_figheight())
    fig.set_figwidth(grid[1] * .8 * fig.get_figwidth())

    for ds in ds_keys:
        i_row = 0
        for p in params:
            ax = plt.subplot2grid(grid, (i_row, i_col))
            ax.hist(R[ds]['combined']['params'][p], bins=100)
            if i_row == grid[0] - 1:
                ax.set_xlabel('Value [a.u.]')
            if i_col == 0:
                ax.set_ylabel('Occurrences [#]')
            ax.set_title('{:s}: {:s}'.format(ds, p))
            i_row += 1
        i_col += 1

    pdf.savefig(fig)
    plt.show(fig)
    plt.close(fig)

In [ ]:
# Plot onset time correlations
pdffile = os.path.join(getOutpath(), '{:s}_onset_correlations.pdf'.format(getTimeStamp()))
with PdfPages(pdffile) as pdf:
    for k in R.keys():
        fig = plt.figure()
        plt.plot([0, 30], [0, 30], 'k-')
        plt.plot(R[k]['combined']['params']['tr'], R[k]['combined']['params']['tg'], '.')
        plt.xlabel('Onset RFP [h]')
        plt.ylabel('Onset GFP [h]')
        plt.title(k)
        pdf.savefig(fig)
        plt.show()
        plt.close()
    

From the [documentation](http://lmfit-py.readthedocs.io/en/latest/model.html#lmfit.model.Model.fit):

If supplied, `weights` will be used to weight the calculated residual so that the quantity minimized in the least-squares sense is `weights*(data - fit)`. `weights` must be an `ndarray`-like object of same size and shape as `data`.

## Playground
This section contains code that was/is used for developing ideas.

In [ ]:
# Degradation rate ratio
def plotHistograms(maxH):
    Rkeys = sorted(R.keys())
    for ds in Rkeys:
        #deltg = R[ds]['green']['params']['deltg']
        #deltr = R[ds]['red']['params']['deltr']
        deltg = R[ds]['combined']['params']['deltg']
        deltr = R[ds]['combined']['params']['deltr']
        quot = deltg / deltr

        fig = plt.figure()
        plt.hist(quot, bins=150, range=(0, maxH))
        plt.title(ds)
        plt.xlabel('$\delta_\mathrm{green} / \delta_\mathrm{red}$ [a.u.]')
        plt.ylabel('Occurrences [#]')
        plt.show(fig)
        plt.close(fig)

wdg.interact(plotHistograms, maxH=wdg.IntSlider(
    value=100, min=0, max=1000, step=10, description='Histogram maximum', continuous_update=False));

In [ ]:
# Fit distribution to degradation rate quotient histograms
def gamma(x, p=2, b=1, s=10):
    return s * b**p * x**(p-1) * np.exp(-b * x) / sc.special.gamma(p)

def gamma2(x, p1=1.9, p2=2.1, b1=0.9, b2=1.1, s1=10, s2=10):
    return gamma(x, p1, b1, s1) + gamma(x, p2, b2, s2)

def weibull(x, lmbd=.2, k=2, s=10):
    return s * lmbd * k * (lmbd * x)**(k - 1) * np.exp(- (lmbd * x)**k)

def weibull2(x, lmbd1=.15, lmbd2=.25, k1=1.9, k2=2.1, s1=10, s2=10):
    return weibull(x, lmbd=lmbd1, k=k1, s=s1) + weibull(x, lmbd=lmbd2, k=k2, s=s2)

# Define models
model_gamma = lm.Model(gamma)
model_gamma.set_param_hint(name='p', min=.01)
model_gamma.set_param_hint(name='b', min=.01)
model_gamma.set_param_hint(name='s', min=1)

model_gamma2 = lm.Model(gamma2)
model_gamma2.set_param_hint(name='p1', min=.01)
model_gamma2.set_param_hint(name='p2', min=.01)
model_gamma2.set_param_hint(name='b1', min=.01)
model_gamma2.set_param_hint(name='b2', min=.01)
model_gamma2.set_param_hint(name='s1', min=1)
model_gamma2.set_param_hint(name='s2', min=1)

model_weibull = lm.Model(weibull)
model_weibull.set_param_hint(name='lmbd', min=.001)
model_weibull.set_param_hint(name='k', min=.001, max=5)
model_weibull.set_param_hint(name='s', min=1)

model_weibull2 = lm.Model(weibull2)
model_weibull2.set_param_hint(name='lmbd1', min=.001)
model_weibull2.set_param_hint(name='lmbd2', min=.001)
model_weibull2.set_param_hint(name='k1', min=.001, max=5)
model_weibull2.set_param_hint(name='k2', min=.001, max=5)
model_weibull2.set_param_hint(name='s1', min=1)
model_weibull2.set_param_hint(name='s2', min=1)

maxH = 40

with PdfPages(os.path.join(getOutpath(), '{:s}_degradation_distribution.pdf'.format(getTimeStamp()))) as pdf:
    for ds in sorted(R.keys()):
        # Calculate degradation rate quotient
        deltg = R[ds]['combined']['params']['deltg']
        deltr = R[ds]['combined']['params']['deltr']
        quot = deltg / deltr

        # Create histogram
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        hist_val, hist_edg = ax.hist(quot, bins=70, range=(0, maxH), label='Histogram')[:2]
        hist_ctr = (hist_edg[:-1] + hist_edg[1:]) / 2

        # Fit models
        result_g = model_gamma.fit(hist_val, x=hist_ctr)
        result_g2 = model_gamma2.fit(hist_val, x=hist_ctr)
        result_w = model_weibull.fit(hist_val, x=hist_ctr)
        result_w2 = model_weibull2.fit(hist_val, x=hist_ctr)

        # Select models
        #print('gamma: {}'.format(result_g.chisqr))
        #print('gamma2: {}'.format(result_g2.chisqr))
        #print('weibull: {}'.format(result_w.chisqr))
        #print('weibull2: {}'.format(result_w2.chisqr))

        if result_g2.chisqr < .7 * result_g.chisqr:
            res_g = result_g2
            name_g = 'gamma2'
        else:
            res_g = result_g
            name_g = 'gamma'

        if result_w2.chisqr < .7 * result_w.chisqr:
            res_w = result_w2
            name_w = 'weibull2'
        else:
            res_w = result_w
            name_w = 'weibull'

        # Plot models
        x = np.linspace(.1, 5, 100)
        ax.plot(hist_ctr, res_g.best_fit, '-', label=name_g, color='orange')
        ax.plot(hist_ctr, res_w.best_fit, '-', label=name_w, color='magenta')
        ax.legend()
        ax.set_xlabel('$\delta_\mathrm{green} / \delta_\mathrm{red}$ [a.u.]')
        ax.set_ylabel('Counts [#]')
        ax.set_title(ds)

        # Print fit reports
        rep = res_g.fit_report(show_correl=False) + '\n' + res_w.fit_report(show_correl=False)
        ax = fig.add_subplot(1, 2, 2)
        ax.set_axis_off()
        ax.text(0, 1, rep, ha='left', va='top', family='monospace', size=5.5)

        # Display, save and close figure
        plt.show(fig)
        pdf.savefig(fig)
        plt.close(fig)

In [ ]:
# Scatter plot of degradation rates
Rkeys = sorted(R.keys())
for ds in Rkeys:
    deltg = R[ds]['combined']['params']['deltg']
    deltr = R[ds]['combined']['params']['deltr']

    fig = plt.figure()
    h = plt.plot(deltg, deltr, '.')
    plt.title(ds)
    plt.xlabel('$\delta_\mathrm{green}$ [a.u.]')
    plt.ylabel('$\delta_\mathrm{red}$ [a.u.]')
    plt.xscale('log')
    plt.yscale('log')
    plt.show(fig)
    plt.close(fig)

In [ ]:
# Plot violins
pn_both = ['m', 'ktl']
pn_red = ['tr', 'kmr', 'betr', 'deltr', 'offr']
pn_green = ['tg', 'kmg', 'betg', 'deltg', 'offg']
grid = (2, 1+max(len(pn_red), len(pn_green)))

ts = getTimeStamp()

for ds in R.keys():

    pdffile = os.path.join(getOutpath(), '{:s}_violins_{:s}.pdf'.format(ts, ds.replace(' ', '_')))
    with PdfPages(pdffile) as pdf:

        for i in R[ds]['combined']['params'].index:
            fig = plt.figure()
            fig.suptitle('{:s} (combined fit) #{:03d}'.format(ds, i))

            # Plot combined parameters
            for prm in range(len(pn_both)):
                ax = plt.subplot2grid(grid, (prm, 0))
                label = pn_both[prm]
                data = R[ds]['combined']['params'][label].values
                clr_face = '#0000ff55'
                clr_edge = '#000099ff'
                plotViolin(ax, data, label, clr_face, clr_edge, data[i])

            # Plot green parameters
            for prm in range(len(pn_green)):
                ax = plt.subplot2grid(grid, (0, prm+1))
                label = pn_green[prm]
                data = R[ds]['combined']['params'][label].values
                clr_face = '#00ff0055'
                clr_edge = '#009900ff'
                plotViolin(ax, data, label, clr_face, clr_edge, data[i])

            # Plot red parameters
            for prm in range(len(pn_red)):
                ax = plt.subplot2grid(grid, (1, prm+1))
                label = pn_red[prm]
                data = R[ds]['combined']['params'][label].values
                clr_face = '#ff000055'
                clr_edge = '#990000ff'
                plotViolin(ax, data, label, clr_face, clr_edge, data[i])
        
            pdf.savefig(fig)
            plt.show(fig)
            plt.close(fig)
